In [ ]:
## PRIMERA PARTE 
# Load keypoint sequence
seq = np.load("../app/debug_sequence.npy")

# Load scaler and normalize
with open("../app/scaler.pkl", "rb") as f:
    scaler = pickle.load(f)
seq = (seq - scaler['mean']) / scaler['std']
tensor = torch.from_numpy(seq.astype(np.float32)).unsqueeze(0)

# Load model
#model = SignLSTM(input_size=126, hidden_size=128, num_layers=2, num_classes=15)
#model.load_state_dict(torch.load("../app/model.pth", map_location="cpu"))
model.load_state_dict(torch.load("best_model.pt", map_location="cpu"))
model.eval()

# Predict
with torch.no_grad():
    logits = model(tensor)
    probs = torch.softmax(logits, dim=1).numpy()[0]
    predicted_idx = int(probs.argmax())

# Load label encoder
with open("../app/label_encoder.pkl", "rb") as f:
    le = pickle.load(f)

print("🔍 Prediction:", le.inverse_transform([predicted_idx])[0])
print("Probabilities:")
for i, p in enumerate(probs):
    print(f"  {le.inverse_transform([i])[0]}: {p:.4f}")

#### output app 
```
✅ Keypoint sequence saved to debug_sequence.npy
Raw logits: tensor([[-0.9475,  0.4579,  1.5646, -0.6141,  7.0239,  0.4305, -1.7777, -2.6337,
         -2.9380,  0.2447, -1.1950, -1.7126,  1.9589, -0.2472, -0.5548]])
Probabilities:
  bad: 0.0003
  bye: 0.0014
  good: 0.0042
  hello: 0.0005
  help: 0.9831
  love: 0.0013
  me: 0.0001
  no: 0.0001
  please: 0.0000
  sorry: 0.0011
  stop: 0.0003
  thank_you: 0.0002
  world: 0.0062
  yes: 0.0007
  you: 0.0005
Prediction: help | Confidence: 0.9830814003944397
```

#### output sequencia app en NB

```
🔍 Prediction: love
Probabilities:
  bad: 0.0082
  bye: 0.0220
  good: 0.0231
  hello: 0.2244
  help: 0.0107
  love: 0.6110
  me: 0.0049
  no: 0.0004
  please: 0.0449
  sorry: 0.0044
  stop: 0.0030
  thank_you: 0.0017
  world: 0.0022
  yes: 0.0386
  you: 0.0006
```

la misma secuencia .npy genera dos predicciones diferentes en la app y en el notebook. Eso confirma que no es un problema de datos ni del modelo entrenado en sí, sino de inconsistencias entre los entornos o la forma en que se cargan los modelos y artefactos.
(La prediccion correcta es Love)

In [27]:
import torch
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
import sys
sys.path.append('../models')
from lstm import SignLSTM

# ─── LOAD SEQUENCE ────────────────────────────────────────────────
seq = np.load("../app/debug_sequence2.npy") 
#seq = np.load("../data/keypoints_augmented/good/good_1_aug0.npy")

# ─── LOAD SCALER ──────────────────────────────────────────────────
with open("../app/scaler.pkl", "rb") as f:
    scaler = pickle.load(f)
seq = (seq - scaler['mean']) / scaler['std']
tensor = torch.from_numpy(seq.astype(np.float32)).unsqueeze(0)

# ─── LOAD MODEL ───────────────────────────────────────────────────
# Debes pasar los parámetros correctos
model = SignLSTM(input_size=126, hidden_size=128, num_layers=2, num_classes=15)
model.load_state_dict(torch.load("best_model.pt", map_location="cpu"))
model.eval()

# ─── INFERENCE ────────────────────────────────────────────────────
with torch.no_grad():
    logits = model(tensor)
    probs = torch.softmax(logits, dim=1).numpy()[0]
    predicted_idx = int(probs.argmax())

# ─── LABELS ───────────────────────────────────────────────────────
with open("../app/label_encoder.pkl", "rb") as f:
    le = pickle.load(f)

# ─── OUTPUT ───────────────────────────────────────────────────────
print("🔍 Prediction from debug_sequence2.npy:", le.inverse_transform([predicted_idx])[0])
print("Probabilities:")
for i, p in enumerate(probs):
    print(f"  {le.inverse_transform([i])[0]}: {p:.4f}")


🔍 Prediction from debug_sequence2.npy: good
Probabilities:
  bad: 0.0020
  bye: 0.0002
  good: 0.9620
  hello: 0.0001
  help: 0.0053
  love: 0.0006
  me: 0.0190
  no: 0.0005
  please: 0.0007
  sorry: 0.0015
  stop: 0.0057
  thank_you: 0.0006
  world: 0.0009
  yes: 0.0001
  you: 0.0009


````
✅ Keypoint sequence saved to debug_sequence2.npy
Raw logits: tensor([[-1.8610,  1.0745,  0.4421, -1.6416, -0.4929, -3.4781, -2.0141,  8.7493,
         -2.5912, -2.1223,  0.0804,  1.8194, -2.7973,  1.6758, -0.6637]])
Probabilities:
  bad: 0.0000
  bye: 0.0005
  good: 0.0002
  hello: 0.0000
  help: 0.0001
  love: 0.0000
  me: 0.0000
  no: 0.9970
  please: 0.0000
  sorry: 0.0000
  stop: 0.0002
  thank_you: 0.0010
  world: 0.0000
  yes: 0.0008
  you: 0.0001
Prediction: no | Confidence: 0.9970000386238098

````



In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

# Function to visualize keypoints from a .npy file
def visualize_keypoints_hands(npy_file):
    keypoints = np.load(npy_file)  # Load keypoints data
    # Desaplanar si viene en forma (T, 126)
    if keypoints.ndim == 2 and keypoints.shape[1] == 126:
        keypoints = keypoints.reshape((-1, 42, 3))
        
    num_frames = keypoints.shape[0]

    print(f"Visualizing {os.path.basename(npy_file)} ({num_frames} frames)")

    # Create a blank image for visualization
    img_size = 500
    blank_frame = np.ones((img_size, img_size, 3), dtype=np.uint8) * 255

    # Define landmarks (POSE = 33, HAND = 21 per hand)
    POSE_LANDMARKS = 33
    HAND_LANDMARKS = 21
    TOTAL_KEYPOINTS = 2 * HAND_LANDMARKS


    # Animate the keypoints frame by frame
    for frame_idx in range(num_frames):
        img = blank_frame.copy()
        frame_keypoints = keypoints[frame_idx]

        # Normalize & scale keypoints to fit the image
        scaled_keypoints = (frame_keypoints[:, :2] * img_size).astype(int)

        # Draw left hand landmarks
        for i in range(HAND_LANDMARKS):
            x, y = scaled_keypoints[i]
            cv2.circle(img, (x, y), 3, (255, 0, 0), -1)  # Blue for left hand

        # Draw right hand landmarks
        for i in range(HAND_LANDMARKS):
            x, y = scaled_keypoints[HAND_LANDMARKS + i]
            cv2.circle(img, (x, y), 3, (0, 255, 0), -1)  # Green for right hand

        # Show the frame
        cv2.imshow("Keypoints Visualization", img)
        if cv2.waitKey(50) & 0xFF == ord('q'):  # Press 'q' to quit
            break

    cv2.destroyAllWindows()
    print(" Visualization complete.")

In [ ]:
# Folder where keypoint files are stored
debug = "../app/debug_sequence3.npy"
# Visualize the first keypoint file (change index for other files)
visualize_keypoints_hands(debug)


Visualizing debug_sequence3.npy (60 frames)


2025-05-17 18:34:54.859 Python[34464:1584305] +[IMKClient subclass]: chose IMKClient_Modern
2025-05-17 18:34:54.859 Python[34464:1584305] +[IMKInputSession subclass]: chose IMKInputSession_Modern


 Visualization complete.


: 

In [ ]:
#forzar salida de la visualizacion
cv2.waitKey(1)
cv2.destroyAllWindows()

: 

In [15]:
import torch
import numpy as np
import pickle
import os
import sys

# ─── IMPORTAR CLASE DEL MODELO ────────────────────────────────────────────────
sys.path.append('../models')
from lstm import SignLSTM

# ─── CONFIGURACIÓN ────────────────────────────────────────────────────────────
keypoints_folder = "../data/tracking_keypoints/keypoints_augmented/good"
SEQUENCE_LENGTH = 60
model_path = "../code/best_model.pt"
scaler_path = "../app/scaler.pkl"
label_encoder_path = "../app/label_encoder.pkl"

# ─── CARGAR SECUENCIA DE UN .NPY DEL DATASET ──────────────────────────────────
npy_files = [f for f in os.listdir(keypoints_folder) if f.endswith(".npy")]
print(f"Found {len(npy_files)} keypoint files.")

# Seleccionar uno (por ejemplo el número 100)
file_path = os.path.join(keypoints_folder, npy_files[100])
seq = np.load(file_path)

# Validación de forma esperada
if seq.shape[1:] != (42, 3):
    raise ValueError(f"❌ Unexpected shape in {file_path}: {seq.shape}")

# Recortar o rellenar a 30 frames

if seq.shape[0] < SEQUENCE_LENGTH:
    pad_width = SEQUENCE_LENGTH - seq.shape[0]
    seq = np.pad(seq, ((0, pad_width), (0, 0), (0, 0)), mode='constant')
            
else:
    seq = seq[:SEQUENCE_LENGTH]

# Aplanar a (30, 126)
seq = seq.reshape(SEQUENCE_LENGTH, -1)

# ─── NORMALIZAR ───────────────────────────────────────────────────────────────
with open(scaler_path, "rb") as f:
    scaler = pickle.load(f)
seq = (seq - scaler['mean']) / scaler['std']
tensor = torch.from_numpy(seq.astype(np.float32)).unsqueeze(0)

# ─── CARGAR MODELO ────────────────────────────────────────────────────────────
model = SignLSTM(input_size=126, hidden_size=128, num_layers=2, num_classes=15)
model.load_state_dict(torch.load(model_path, map_location="cpu"))
model.eval()

# ─── INFERENCIA ───────────────────────────────────────────────────────────────
with torch.no_grad():
    logits = model(tensor)
    probs = torch.softmax(logits, dim=1).numpy()[0]
    predicted_idx = int(probs.argmax())

# ─── DECODIFICAR PREDICCIÓN ───────────────────────────────────────────────────
with open(label_encoder_path, "rb") as f:
    le = pickle.load(f)

print(f"\n🔍 Prediction for file: {npy_files[100]}")
print(f"👉 Predicted sign: {le.inverse_transform([predicted_idx])[0]}")
print("📊 Probabilities:")
for i, p in enumerate(probs):
    print(f"  {le.inverse_transform([i])[0]}: {p:.4f}")


Found 360 keypoint files.

🔍 Prediction for file: good_31_aug0.npy
👉 Predicted sign: hello
📊 Probabilities:
  bad: 0.0005
  bye: 0.0093
  good: 0.0010
  hello: 0.6619
  help: 0.0010
  love: 0.0016
  me: 0.0004
  no: 0.0032
  please: 0.0107
  sorry: 0.0020
  stop: 0.0019
  thank_you: 0.2691
  world: 0.0045
  yes: 0.0322
  you: 0.0009
